# CSCN8020 - Assignment 1:


# Problem 1
## Pick-and-Place Robot MDP Design

### Problem Statement
Design a reinforcement learning problem as an MDP for controlling a robot arm in a repetitive pick-and-place task. The goal is to learn movements that are fast and smooth through direct motor control.

---

### MDP Components

#### States (S)
The state space captures all information necessary for effective control decisions:

- **Joint angles**: θ = (θ₁, θ₂, ..., θₙ) for each joint in the arm
- **Joint angular velocities**: θ̇ = (θ̇₁, θ̇₂, ..., θ̇ₙ) for each joint
- **End-effector position**: (x, y, z) in 3D space
- **Gripper state**: (open/closed)
- **Object status**: (held/not held)
- **Target position**: Current pick or place location

**Reasoning**: Positions tell us where the arm is located, while velocities are essential for achieving smooth motion and avoiding jerky movements. Task status information (gripper and object states) helps the agent know whether to pick or place. Since the objective emphasizes smooth movements, velocity information is crucial for the learning process.

---

#### Actions (A)
The action space consists of continuous motor commands:

- **Motor torques**: τ = (τ₁, τ₂, ..., τₙ) applied to each joint
- **Gripper command**: (open/close/hold)

**Reasoning**: Since we're controlling motors directly, actions should be torque commands or voltage signals to each motor. Continuous actions allow for smooth, precise control, which directly aligns with the smoothness objective. This low-level control gives the agent the flexibility to learn optimal trajectories.

---

#### Rewards (R)
A composite reward function that balances speed, smoothness, and task completion:

**R(s, a, s') = R_task + R_speed + R_smoothness + R_energy**

**Components:**

1. **Task Rewards (R_task)**:
   - +100 for successful object pick
   - +100 for successful object placement
   - -1 per timestep (encourages faster task completion)

2. **Speed Rewards (R_speed)**:
   - -0.1 × time_elapsed (encourages faster movements)

3. **Smoothness Rewards (R_smoothness)**:
   - -α × ||a - a_prev||² (penalizes sudden changes in motor commands)

4. **Energy Rewards (R_energy)**:
   - -β × ||a||² (penalizes excessive motor torque, encourages efficiency)

**Additional Penalties:**
- -50 for dropping the object
- -10 for collisions with workspace boundaries
- -5 for excessive joint velocities

**Reasoning**: The reward structure directly addresses the learning objectives. Task completion rewards ensure the robot fulfills its function. Time penalties encourage speed. Smoothness penalties (penalizing large changes between consecutive actions) promote smooth trajectories by discouraging abrupt motor command changes. Energy penalties prevent wasteful movements and protect the hardware. This multi-objective reward naturally guides the agent toward desired behaviors.

---

#### Transition Dynamics (P)
The transition function P(s'|s, a) is determined by the physics of the robot arm:

- **Forward dynamics equations**: How applied torques affect joint accelerations
- **Kinematic constraints**: Joint limits and workspace boundaries
- **Contact dynamics**: Gripper-object interaction physics
- **Environmental factors**: Gravity, friction, and inertia

**Reasoning**: In model-free RL, we may not explicitly model these dynamics, but the environment simulator must encode the physical laws governing robot motion. The deterministic or stochastic nature depends on factors like sensor noise and environmental uncertainty.

---

### Additional Design Considerations

1. **Discount Factor (γ)**: Use γ ≈ 0.95-0.99 to balance immediate rewards (speed) with long-term trajectory smoothness

2. **Episode Structure**: Each episode starts with the object at the pick location and ends when successfully placed at the target or after a maximum timeout

3. **State Representation**: For real implementation, continuous states would likely use function approximation (neural networks) rather than discretization

4. **Action Space**: Continuous control is preferred, potentially using algorithms like DDPG, TD3, or SAC

---

### Talking Points:

For this pick-and-place robot task, I've designed an MDP that enables learning of fast and smooth movements through direct motor control.

The **state space** includes joint angles, joint velocities, end-effector position, gripper state, and object status. Including velocities is critical because smooth motion depends on understanding movement dynamics, not just positions.

**Actions** are continuous motor torques applied to each joint, plus gripper commands. This low-level control allows the agent to learn precise, smooth trajectories.

The **reward function** balances multiple objectives: task completion bonuses (+100 for successful pick/place), time penalties to encourage speed (-1 per timestep), smoothness penalties that discourage abrupt motor command changes (-α × ||a - a_prev||²), and energy penalties to prevent excessive torques. This structure directly incentivizes the desired behavior.

**Transition dynamics** follow the physical laws governing the robot arm—how torques affect joint motion and object interactions.

This formulation provides sufficient information for learning while the reward design naturally guides the agent toward efficient, smooth pick-and-place behaviors.

---




# Problem 2
## Value Iteration on 2×2 Gridworld (Manual Calculation)

### Problem Statement
Perform two iterations of Value Iteration for a 2×2 gridworld environment. Show the step-by-step process (without code) including policy evaluation and policy improvement.

---

### Environment Setup

#### Grid Layout
```
[s1: R=5]   [s2: R=10]
[s3: R=1]   [s4: R=2]
```

#### Environment Characteristics

**State Space (S)**: {s1, s2, s3, s4}

**Action Space (A)**: {up, down, left, right}

**Initial Policy (π)**: For all states, π(up|s) = 1

**Transition Probabilities P(s'|s, a)**:
- If the action is valid (does not run into a wall), the transition is deterministic
- Otherwise, s' = s (agent stays in current state)

**Rewards R(s)**:
- R(s1) = 5 for all actions a
- R(s2) = 10 for all actions a
- R(s3) = 1 for all actions a
- R(s4) = 2 for all actions a

**Discount Factor**: γ = 0.9 (assumed)

---

### Value Iteration Algorithm

The Bellman optimality equation used for value iteration:

\[ V(s) = \max_a \sum_{s'} P(s'|s,a) [R(s) + \gamma V(s')] \]

For deterministic transitions, this simplifies to:

\[ V(s) = \max_a [R(s) + \gamma V(s')] \]

---

## Iteration 0: Initial Value Function

All states start with zero value:

| State | V₀(s) |
|-------|-------|
| s1    | 0.0   |
| s2    | 0.0   |
| s3    | 0.0   |
| s4    | 0.0   |

**Grid Visualization:**
```
[0.0]  [0.0]
[0.0]  [0.0]
```

---

## Iteration 1: First Value Update

### State s1 (Top-Left, R=5)

Calculate value for each action:

- **up**: Hits wall → stays at s1
  - V = R(s1) + γ × V₀(s1) = 5 + 0.9(0) = **5.0**

- **down**: Moves to s3
  - V = R(s1) + γ × V₀(s3) = 5 + 0.9(0) = **5.0**

- **left**: Hits wall → stays at s1
  - V = R(s1) + γ × V₀(s1) = 5 + 0.9(0) = **5.0**

- **right**: Moves to s2
  - V = R(s1) + γ × V₀(s2) = 5 + 0.9(0) = **5.0**

**V₁(s1) = max(5.0, 5.0, 5.0, 5.0) = 5.0**

---

### State s2 (Top-Right, R=10)

Calculate value for each action:

- **up**: Hits wall → stays at s2
  - V = R(s2) + γ × V₀(s2) = 10 + 0.9(0) = **10.0**

- **down**: Moves to s4
  - V = R(s2) + γ × V₀(s4) = 10 + 0.9(0) = **10.0**

- **left**: Moves to s1
  - V = R(s2) + γ × V₀(s1) = 10 + 0.9(0) = **10.0**

- **right**: Hits wall → stays at s2
  - V = R(s2) + γ × V₀(s2) = 10 + 0.9(0) = **10.0**

**V₁(s2) = max(10.0, 10.0, 10.0, 10.0) = 10.0**

---

### State s3 (Bottom-Left, R=1)

Calculate value for each action:

- **up**: Moves to s1
  - V = R(s3) + γ × V₀(s1) = 1 + 0.9(0) = **1.0**

- **down**: Hits wall → stays at s3
  - V = R(s3) + γ × V₀(s3) = 1 + 0.9(0) = **1.0**

- **left**: Hits wall → stays at s3
  - V = R(s3) + γ × V₀(s3) = 1 + 0.9(0) = **1.0**

- **right**: Moves to s4
  - V = R(s3) + γ × V₀(s4) = 1 + 0.9(0) = **1.0**

**V₁(s3) = max(1.0, 1.0, 1.0, 1.0) = 1.0**

---

### State s4 (Bottom-Right, R=2)

Calculate value for each action:

- **up**: Moves to s2
  - V = R(s4) + γ × V₀(s2) = 2 + 0.9(0) = **2.0**

- **down**: Hits wall → stays at s4
  - V = R(s4) + γ × V₀(s4) = 2 + 0.9(0) = **2.0**

- **left**: Moves to s3
  - V = R(s4) + γ × V₀(s3) = 2 + 0.9(0) = **2.0**

- **right**: Hits wall → stays at s4
  - V = R(s4) + γ × V₀(s4) = 2 + 0.9(0) = **2.0**

**V₁(s4) = max(2.0, 2.0, 2.0, 2.0) = 2.0**

---

### Updated Value Function After Iteration 1

| State | V₁(s) |
|-------|-------|
| s1    | 5.0   |
| s2    | 10.0  |
| s3    | 1.0   |
| s4    | 2.0   |

**Grid Visualization:**
```
[5.0]   [10.0]
[1.0]   [2.0]
```

**Observation**: After the first iteration, the values equal the immediate rewards since all successor state values were zero.

---

## Iteration 2: Second Value Update

Now we use V₁ values from Iteration 1 to update each state.

### State s1 (Top-Left, R=5)

Calculate value for each action:

- **up**: Hits wall → stays at s1
  - V = R(s1) + γ × V₁(s1) = 5 + 0.9(5.0) = 5 + 4.5 = **9.5**

- **down**: Moves to s3
  - V = R(s1) + γ × V₁(s3) = 5 + 0.9(1.0) = 5 + 0.9 = **5.9**

- **left**: Hits wall → stays at s1
  - V = R(s1) + γ × V₁(s1) = 5 + 0.9(5.0) = 5 + 4.5 = **9.5**

- **right**: Moves to s2
  - V = R(s1) + γ × V₁(s2) = 5 + 0.9(10.0) = 5 + 9.0 = **14.0** ✓

**V₂(s1) = max(9.5, 5.9, 9.5, 14.0) = 14.0**

**Best Action**: right (move to s2)

---

### State s2 (Top-Right, R=10)

Calculate value for each action:

- **up**: Hits wall → stays at s2
  - V = R(s2) + γ × V₁(s2) = 10 + 0.9(10.0) = 10 + 9.0 = **19.0** ✓

- **down**: Moves to s4
  - V = R(s2) + γ × V₁(s4) = 10 + 0.9(2.0) = 10 + 1.8 = **11.8**

- **left**: Moves to s1
  - V = R(s2) + γ × V₁(s1) = 10 + 0.9(5.0) = 10 + 4.5 = **14.5**

- **right**: Hits wall → stays at s2
  - V = R(s2) + γ × V₁(s2) = 10 + 0.9(10.0) = 10 + 9.0 = **19.0** ✓

**V₂(s2) = max(19.0, 11.8, 14.5, 19.0) = 19.0**

**Best Actions**: up or right (both keep agent in s2 or hit wall)

---

### State s3 (Bottom-Left, R=1)

Calculate value for each action:

- **up**: Moves to s1
  - V = R(s3) + γ × V₁(s1) = 1 + 0.9(5.0) = 1 + 4.5 = **5.5** ✓

- **down**: Hits wall → stays at s3
  - V = R(s3) + γ × V₁(s3) = 1 + 0.9(1.0) = 1 + 0.9 = **1.9**

- **left**: Hits wall → stays at s3
  - V = R(s3) + γ × V₁(s3) = 1 + 0.9(1.0) = 1 + 0.9 = **1.9**

- **right**: Moves to s4
  - V = R(s3) + γ × V₁(s4) = 1 + 0.9(2.0) = 1 + 1.8 = **2.8**

**V₂(s3) = max(5.5, 1.9, 1.9, 2.8) = 5.5**

**Best Action**: up (move to s1)

---

### State s4 (Bottom-Right, R=2)

Calculate value for each action:

- **up**: Moves to s2
  - V = R(s4) + γ × V₁(s2) = 2 + 0.9(10.0) = 2 + 9.0 = **11.0** ✓

- **down**: Hits wall → stays at s4
  - V = R(s4) + γ × V₁(s4) = 2 + 0.9(2.0) = 2 + 1.8 = **3.8**

- **left**: Moves to s3
  - V = R(s4) + γ × V₁(s3) = 2 + 0.9(1.0) = 2 + 0.9 = **2.9**

- **right**: Hits wall → stays at s4
  - V = R(s4) + γ × V₁(s4) = 2 + 0.9(2.0) = 2 + 1.8 = **3.8**

**V₂(s4) = max(11.0, 3.8, 2.9, 3.8) = 11.0**

**Best Action**: up (move to s2)

---

### Updated Value Function After Iteration 2

| State | V₂(s) | Best Action |
|-------|-------|-------------|
| s1    | 14.0  | right       |
| s2    | 19.0  | up/right    |
| s3    | 5.5   | up          |
| s4    | 11.0  | up          |

**Grid Visualization:**
```
[14.0]  [19.0]
[5.5]   [11.0]
```

---

## Summary Table

| State | V₀ | V₁  | V₂   | Optimal Action |
|-------|-----|-----|------|----------------|
| s1    | 0.0 | 5.0 | 14.0 | right          |
| s2    | 0.0 | 10.0| 19.0 | up/right       |
| s3    | 0.0 | 1.0 | 5.5  | up             |
| s4    | 0.0 | 2.0 | 11.0 | up             |

---

## Key Observations

1. **Value Propagation**: Values increase with each iteration as the agent learns to account for future rewards through the discount factor.

2. **Optimal Behavior Emerging**: After two iterations, the policy is gravitating toward state s2 (highest immediate reward of 10), with:
   - s1 → right (to s2)
   - s3 → up (to s1, then eventually to s2)
   - s4 → up (directly to s2)
   - s2 → stay (up or right keeps agent in high-reward state)

3. **Convergence**: The values are still changing, indicating that more iterations would be needed to reach the optimal value function V*.

4. **Policy Improvement**: The initial policy (all states go up) was suboptimal. After two iterations of value iteration, we've identified better actions that maximize expected cumulative reward.

---

## Mathematical Process

The value iteration process follows this pattern:

**Iteration 1**: Values reflect only immediate rewards
\[ V₁(s) = R(s) + \gamma × 0 = R(s) \]

**Iteration 2**: Values incorporate one-step lookahead
\[ V₂(s) = \max_a [R(s) + \gamma × V₁(s')] \]

**Subsequent Iterations**: Continue until convergence
\[ V_{k+1}(s) = \max_a [R(s) + \gamma × V_k(s')] \]

The algorithm converges when the maximum change in value function falls below a threshold:
\[ \max_s |V_{k+1}(s) - V_k(s)| < \theta \]

---
## Talking Points:

I performed two iterations of Value Iteration on a 2×2 gridworld with rewards of 5, 10, 1, and 2 for states s1, s2, s3, and s4 respectively, using a discount factor of 0.9.

**Iteration 1** produced values equal to the immediate rewards (5.0, 10.0, 1.0, 2.0) since all successor states started at zero. 
**Iteration 2** showed significant value propagation: s1 increased to 14.0, s2 to 19.0, s3 to 5.5, and s4 to 11.0.

The optimal policy emerged clearly all states gravitate toward s2, which has the highest immediate reward. State s1's best action is to move right to s2, s3 should move up toward s1 (then to s2), and s4 should move up directly to s2. State s2 itself benefits from staying put.

This demonstrates how value iteration uses the Bellman optimality equation to propagate future reward information backward through the state space, allowing the agent to discover that maximizing long-term value means navigating to and remaining in high-reward regions.

---

## Problem 3 

PROBLEM 3: 5x5 Gridworld with Value Iteration
GridWorld Environment Initialized
Size: 5x5
Goal State: (4, 4) (Reward: +10)
Grey States: [(1, 2), (3, 0), (0, 4)] (Reward: -5)
Regular States: All others (Reward: -1)

Reward Matrix:
[[-1. -1. -1. -1. -5.]
 [-1. -1. -5. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-5. -1. -1. -1. -1.]
 [-1. -1. -1. -1. 10.]]

Hyperparameters:
  Discount Factor (γ): 0.9
  Convergence Threshold (θ): 0.0001

STANDARD VALUE ITERATION
Iteration 1: Max Delta = 5.000000
Iteration 9: Max Delta = 0.000000

Converged after 9 iterations!
Time elapsed: 0.003006 seconds

Optimal State-Value Function V*:
Row 0:   -5.70    -5.22    -4.69    -4.10    -7.44
Row 1:   -5.22    -4.69    -8.10    -3.44    -2.71
Row 2:   -4.69    -4.10    -3.44    -2.71    -1.90
Row 3:   -8.10    -3.44    -2.71    -1.90    -1.00
Row 4:   -3.44    -2.71    -1.90    -1.00     0.00

Optimal Policy π*:
Row 0:   →    →    →    ↓    ↓
Row 1:   →    ↓    →    →    ↓
Row 2:   →    →    →    →    ↓
Row 3:   →    →

,Metric,Standard VI,In-Place VI
0,Iterations to Converge,9,9
1,Optimization Time (s),0.003006,0.001000
2,Final Max Delta,0.00000000,0.00000000
3,Avg Time per Iteration (ms),0.3340,0.1112



2. SPEEDUP ANALYSIS
----------------------------------------------------------------------
In-Place Speedup Factor: 3.00x faster
Time Saved: 2.005 ms
Iteration Difference: 0 iterations

3. COMPUTATIONAL COMPLEXITY ANALYSIS
----------------------------------------------------------------------
Standard Value Iteration:
  • Time Complexity: O(k × |S| × |A|) per iteration
  • Space Complexity: O(2|S|) - requires two value arrays
  • Memory: 2 arrays × 25 states = 50 float values
  • Must copy entire array each iteration

In-Place Value Iteration:
  • Time Complexity: O(k × |S| × |A|) per iteration
  • Space Complexity: O(|S|) - requires single value array
  • Memory: 1 array × 25 states = 25 float values
  • No array copying needed
  • Memory savings: 50% reduction

Where: |S| = 25 states, |A| = 4 actions, k = iterations

4. SOLUTION QUALITY VERIFICATION
----------------------------------------------------------------------
Value Functions Match: True
Policies Match: True
Maximum Value D

SUMMARY TABLE: STATE VALUES AND OPTIMAL ACTIONS


,State,Type,Reward,V*(Standard),V*(In-Place),Optimal Action
0,"(0,0)",Regular,-1.0,-5.695,-5.695,Right
1,"(0,1)",Regular,-1.0,-5.217,-5.217,Right
2,"(0,2)",Regular,-1.0,-4.686,-4.686,Right
3,"(0,3)",Regular,-1.0,-4.095,-4.095,Down
4,"(0,4)",Grey,-5.0,-7.439,-7.439,Down
5,"(1,0)",Regular,-1.0,-5.217,-5.217,Right
6,"(1,1)",Regular,-1.0,-4.686,-4.686,Down
7,"(1,2)",Grey,-5.0,-8.095,-8.095,Right
8,"(1,3)",Regular,-1.0,-3.439,-3.439,Right
9,"(1,4)",Regular,-1.0,-2.710,-2.710,Down


In [3]:
#!/usr/bin/python3


import numpy as np
import time
import matplotlib.pyplot as plt
from IPython.display import display
import pandas as pd

# ============================================================================
# GRIDWORLD ENVIRONMENT CLASS (Updated for Problem 3)
# ============================================================================

class GridWorld():
    """
    GridWorld environment for reinforcement learning.
    5x5 grid with goal state, grey penalty states, and regular states.
    """

    def __init__(self, env_size):
        self.env_size = env_size
        # Terminal/Goal state at bottom-right corner
        self.terminal_state = (4, 4)

        # Define the transition probabilities and rewards
        self.actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up
        self.action_description = ["Right", "Left", "Down", "Up"]
        self.action_symbols = ["→", "←", "↓", "↑"]

        # TASK 1: Update reward function as specified in Problem 3
        # Initialize all states with -1 (regular states)
        self.reward = np.ones((self.env_size, self.env_size)) * -1

        # Grey states (unfavorable): -5
        self.grey_states = [(1, 2), (3, 0), (0, 4)]
        for state in self.grey_states:
            self.reward[state] = -5

        # Terminal/Goal state: +10
        self.reward[self.terminal_state] = 10

        print("GridWorld Environment Initialized")
        print(f"Size: {env_size}x{env_size}")
        print(f"Goal State: {self.terminal_state} (Reward: +10)")
        print(f"Grey States: {self.grey_states} (Reward: -5)")
        print(f"Regular States: All others (Reward: -1)")
        print("\nReward Matrix:")
        print(self.reward)

    def step(self, action_index, i, j):
        """
        Returns the next state given the chosen action and current state.

        Args:
            action_index: Index of the action to take (0=Right, 1=Left, 2=Down, 3=Up)
            i, j: Current state coordinates

        Returns:
            next_i, next_j: Next state coordinates
            reward: Reward for the current state
            done: Boolean indicating if terminal state reached
        """
        # Transition Probability Matrix: P(s'|s,a) = 1.0 for single state, 0 otherwise
        action = self.actions[action_index]
        next_i, next_j = i + action[0], j + action[1]

        # If action leads outside grid, stay in current state
        if not self.is_valid_state(next_i, next_j):
            next_i, next_j = i, j

        done = self.is_terminal_state(next_i, next_j)
        reward = self.reward[i, j]  # Reward based on current state

        return next_i, next_j, reward, done

    def is_valid_state(self, i, j):
        """Checks if a state is within the acceptable bounds of the environment."""
        valid = 0 <= i < self.env_size and 0 <= j < self.env_size
        return valid

    def is_terminal_state(self, i, j):
        """Returns True if the state is a terminal state."""
        return (i, j) == self.terminal_state

    def get_size(self):
        """Returns the size of the environment."""
        return self.env_size

    def get_actions(self):
        """Returns the list of available actions."""
        return self.actions


# ============================================================================
# TASK 1: STANDARD VALUE ITERATION
# ============================================================================

def value_iteration_standard(env, gamma=0.9, theta=0.0001, max_iterations=1000):
    """
    Standard Value Iteration algorithm using two arrays (synchronous updates).

    Algorithm:
    1. Initialize V(s) = 0 for all states
    2. Repeat until convergence:
       - For each state s:
         - Calculate Q(s,a) for all actions a
         - V_new(s) = max_a Q(s,a)
       - Check convergence: if max|V_new - V_old| < theta, stop
    3. Extract optimal policy: π(s) = argmax_a Q(s,a)

    Args:
        env: GridWorld environment
        gamma: Discount factor (default 0.9)
        theta: Convergence threshold (default 0.0001)
        max_iterations: Maximum number of iterations

    Returns:
        V: Optimal value function
        policy: Optimal policy (action indices)
        iterations: Number of iterations to converge
        elapsed_time: Time taken to converge
        convergence_history: List of delta values per iteration
    """
    # Initialize value function to zeros
    V = np.zeros((env.env_size, env.env_size))
    policy = np.zeros((env.env_size, env.env_size), dtype=int)

    iterations = 0
    convergence_history = []
    start_time = time.time()

    print("\n" + "="*70)
    print("STANDARD VALUE ITERATION")
    print("="*70)

    for iteration in range(max_iterations):
        # Create a copy of the current value function (synchronous update)
        V_old = V.copy()
        delta = 0  # Track maximum change in value function

        # Update each state
        for i in range(env.env_size):
            for j in range(env.env_size):
                # Skip terminal state (value is fixed)
                if env.is_terminal_state(i, j):
                    V[i, j] = 0  # Terminal state value is 0
                    continue

                # Calculate action values Q(s,a) for all actions
                action_values = []
                for action_idx in range(len(env.actions)):
                    # Get next state and reward
                    next_i, next_j, reward, done = env.step(action_idx, i, j)

                    # Bellman equation: Q(s,a) = R(s) + γ * V(s')
                    q_value = reward + gamma * V_old[next_i, next_j]
                    action_values.append(q_value)

                # Update value with maximum action value
                V[i, j] = max(action_values)

                # Update policy with best action
                policy[i, j] = np.argmax(action_values)

                # Track convergence
                delta = max(delta, abs(V[i, j] - V_old[i, j]))

        iterations += 1
        convergence_history.append(delta)

        # Print progress every 10 iterations
        if iteration % 10 == 0 or delta < theta:
            print(f"Iteration {iteration + 1}: Max Delta = {delta:.6f}")

        # Check convergence
        if delta < theta:
            print(f"\nConverged after {iterations} iterations!")
            break

    elapsed_time = time.time() - start_time
    print(f"Time elapsed: {elapsed_time:.6f} seconds")

    return V, policy, iterations, elapsed_time, convergence_history


# ============================================================================
# TASK 2: IN-PLACE VALUE ITERATION
# ============================================================================

def value_iteration_inplace(env, gamma=0.9, theta=0.0001, max_iterations=1000):
    """
    In-Place Value Iteration algorithm using single array (asynchronous updates).

    Difference from standard VI:
    - Uses a single value array instead of two
    - Updates are immediately reflected in subsequent calculations within the same iteration
    - More memory efficient: O(|S|) vs O(2|S|)
    - Can converge faster due to immediate propagation of updates

    Args:
        env: GridWorld environment
        gamma: Discount factor (default 0.9)
        theta: Convergence threshold (default 0.0001)
        max_iterations: Maximum number of iterations

    Returns:
        V: Optimal value function
        policy: Optimal policy (action indices)
        iterations: Number of iterations to converge
        elapsed_time: Time taken to converge
        convergence_history: List of delta values per iteration
    """
    # Initialize value function to zeros
    V = np.zeros((env.env_size, env.env_size))
    policy = np.zeros((env.env_size, env.env_size), dtype=int)

    iterations = 0
    convergence_history = []
    start_time = time.time()

    print("\n" + "="*70)
    print("IN-PLACE VALUE ITERATION")
    print("="*70)

    for iteration in range(max_iterations):
        delta = 0  # Track maximum change in value function

        # Update each state (updates are immediately used)
        for i in range(env.env_size):
            for j in range(env.env_size):
                # Skip terminal state (value is fixed)
                if env.is_terminal_state(i, j):
                    V[i, j] = 0  # Terminal state value is 0
                    continue

                # Store old value for convergence check
                v_old = V[i, j]

                # Calculate action values Q(s,a) for all actions
                action_values = []
                for action_idx in range(len(env.actions)):
                    # Get next state and reward
                    next_i, next_j, reward, done = env.step(action_idx, i, j)

                    # Bellman equation: Q(s,a) = R(s) + γ * V(s')
                    # NOTE: V is updated in-place, so we use current values
                    q_value = reward + gamma * V[next_i, next_j]
                    action_values.append(q_value)

                # Update value with maximum action value (immediately)
                V[i, j] = max(action_values)

                # Update policy with best action
                policy[i, j] = np.argmax(action_values)

                # Track convergence
                delta = max(delta, abs(V[i, j] - v_old))

        iterations += 1
        convergence_history.append(delta)

        # Print progress every 10 iterations
        if iteration % 10 == 0 or delta < theta:
            print(f"Iteration {iteration + 1}: Max Delta = {delta:.6f}")

        # Check convergence
        if delta < theta:
            print(f"\nConverged after {iterations} iterations!")
            break

    elapsed_time = time.time() - start_time
    print(f"Time elapsed: {elapsed_time:.6f} seconds")

    return V, policy, iterations, elapsed_time, convergence_history


# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def print_value_function(V, env):
    """Display the value function in a formatted grid."""
    print("\nOptimal State-Value Function V*:")
    print("="*70)
    for i in range(env.env_size):
        row_str = "  ".join([f"{V[i, j]:7.2f}" for j in range(env.env_size)])
        print(f"Row {i}: {row_str}")
    print("="*70)


def print_policy(policy, env):
    """Display the optimal policy in a formatted grid with action symbols."""
    print("\nOptimal Policy π*:")
    print("="*70)

    policy_grid = []
    for i in range(env.env_size):
        row = []
        for j in range(env.env_size):
            if env.is_terminal_state(i, j):
                row.append("G")  # Goal
            else:
                row.append(env.action_symbols[policy[i, j]])
        policy_grid.append(row)

    for i, row in enumerate(policy_grid):
        row_str = "  ".join([f"{cell:>3}" for cell in row])
        print(f"Row {i}: {row_str}")

    print("="*70)
    print("Legend: → (Right), ← (Left), ↓ (Down), ↑ (Up), G (Goal)")
    print("="*70)


def plot_convergence(history_standard, history_inplace):
    """Plot convergence comparison between standard and in-place VI."""
    plt.figure(figsize=(12, 5))

    # Plot 1: Delta over iterations
    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_standard) + 1), history_standard, 
             'b-o', label='Standard VI', linewidth=2, markersize=6)
    plt.plot(range(1, len(history_inplace) + 1), history_inplace, 
             'r-s', label='In-Place VI', linewidth=2, markersize=6)
    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('Max Delta (|V_new - V_old|)', fontsize=12)
    plt.title('Convergence Rate Comparison', fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.yscale('log')

    # Plot 2: Bar chart of iterations
    plt.subplot(1, 2, 2)
    algorithms = ['Standard VI', 'In-Place VI']
    iterations = [len(history_standard), len(history_inplace)]
    colors = ['blue', 'red']
    bars = plt.bar(algorithms, iterations, color=colors, alpha=0.7, edgecolor='black')
    plt.ylabel('Number of Iterations', fontsize=12)
    plt.title('Iterations to Convergence', fontsize=14, fontweight='bold')
    plt.grid(True, axis='y', alpha=0.3)

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.show()


# ============================================================================
# PERFORMANCE COMPARISON
# ============================================================================

def compare_performance(results_standard, results_inplace):
    """
    Compare performance metrics between standard and in-place value iteration.

    Args:
        results_standard: Tuple (V, policy, iterations, time, history)
        results_inplace: Tuple (V, policy, iterations, time, history)
    """
    V_std, policy_std, iters_std, time_std, hist_std = results_standard
    V_inp, policy_inp, iters_inp, time_inp, hist_inp = results_inplace

    print("\n" + "="*70)
    print("PERFORMANCE COMPARISON")
    print("="*70)

    # 1. Convergence Metrics
    print("\n1. CONVERGENCE METRICS")
    print("-"*70)

    comparison_df = pd.DataFrame({
        'Metric': [
            'Iterations to Converge',
            'Optimization Time (s)',
            'Final Max Delta',
            'Avg Time per Iteration (ms)'
        ],
        'Standard VI': [
            iters_std,
            f'{time_std:.6f}',
            f'{hist_std[-1]:.8f}',
            f'{(time_std/iters_std)*1000:.4f}'
        ],
        'In-Place VI': [
            iters_inp,
            f'{time_inp:.6f}',
            f'{hist_inp[-1]:.8f}',
            f'{(time_inp/iters_inp)*1000:.4f}'
        ]
    })

    display(comparison_df)

    # 2. Speedup Analysis
    print("\n2. SPEEDUP ANALYSIS")
    print("-"*70)
    speedup = time_std / time_inp
    time_saved = (time_std - time_inp) * 1000  # Convert to ms
    iter_diff = iters_std - iters_inp

    print(f"In-Place Speedup Factor: {speedup:.2f}x faster")
    print(f"Time Saved: {time_saved:.3f} ms")
    print(f"Iteration Difference: {iter_diff} iterations")

    # 3. Computational Complexity
    print("\n3. COMPUTATIONAL COMPLEXITY ANALYSIS")
    print("-"*70)
    print("Standard Value Iteration:")
    print("  • Time Complexity: O(k × |S| × |A|) per iteration")
    print("  • Space Complexity: O(2|S|) - requires two value arrays")
    print(f"  • Memory: 2 arrays × 25 states = 50 float values")
    print("  • Must copy entire array each iteration")

    print("\nIn-Place Value Iteration:")
    print("  • Time Complexity: O(k × |S| × |A|) per iteration")
    print("  • Space Complexity: O(|S|) - requires single value array")
    print(f"  • Memory: 1 array × 25 states = 25 float values")
    print("  • No array copying needed")
    print("  • Memory savings: 50% reduction")

    print(f"\nWhere: |S| = 25 states, |A| = 4 actions, k = iterations")

    # 4. Solution Quality Verification
    print("\n4. SOLUTION QUALITY VERIFICATION")
    print("-"*70)

    values_match = np.allclose(V_std, V_inp, rtol=1e-5, atol=1e-8)
    policy_match = np.array_equal(policy_std, policy_inp)
    max_value_diff = np.max(np.abs(V_std - V_inp))

    print(f"Value Functions Match: {values_match}")
    print(f"Policies Match: {policy_match}")
    print(f"Maximum Value Difference: {max_value_diff:.10f}")

    if values_match and policy_match:
        print("\n✓ Both algorithms produce IDENTICAL optimal solutions!")

    # 5. Key Observations
    print("\n5. KEY OBSERVATIONS")
    print("-"*70)
    print("✓ In-Place VI is more memory efficient (50% reduction)")
    print("✓ In-Place VI is faster due to:")
    print("  - No array copying overhead")
    print("  - Better cache locality (single array)")
    print("  - Immediate propagation of updated values")
    print("✓ Both algorithms converge to identical optimal solutions")
    print("✓ In-Place VI can converge faster in some state orderings")
    print("✓ For large state spaces, memory savings become critical")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("="*70)
    print("PROBLEM 3: 5x5 Gridworld with Value Iteration")
    print("="*70)

    # Create environment
    env = GridWorld(env_size=5)

    # Set hyperparameters
    GAMMA = 0.9  # Discount factor
    THETA = 0.0001  # Convergence threshold

    print(f"\nHyperparameters:")
    print(f"  Discount Factor (γ): {GAMMA}")
    print(f"  Convergence Threshold (θ): {THETA}")

    # ========================================================================
    # TASK 1: Run Standard Value Iteration
    # ========================================================================
    results_standard = value_iteration_standard(env, gamma=GAMMA, theta=THETA)
    V_standard, policy_standard, iters_std, time_std, history_std = results_standard

    # Display results
    print_value_function(V_standard, env)
    print_policy(policy_standard, env)

    # ========================================================================
    # TASK 2: Run In-Place Value Iteration
    # ========================================================================
    results_inplace = value_iteration_inplace(env, gamma=GAMMA, theta=THETA)
    V_inplace, policy_inplace, iters_inp, time_inp, history_inp = results_inplace

    # Display results
    print_value_function(V_inplace, env)
    print_policy(policy_inplace, env)

    # ========================================================================
    # COMPARE PERFORMANCE
    # ========================================================================
    compare_performance(results_standard, results_inplace)

    # ========================================================================
    # PLOT CONVERGENCE
    # ========================================================================
    plot_convergence(history_std, history_inp)

    # ========================================================================
    # CREATE SUMMARY TABLE
    # ========================================================================
    print("\n" + "="*70)
    print("SUMMARY TABLE: STATE VALUES AND OPTIMAL ACTIONS")
    print("="*70)

    summary_data = []
    for i in range(env.env_size):
        for j in range(env.env_size):
            state_type = "Goal" if (i,j) == env.terminal_state else                         "Grey" if (i,j) in env.grey_states else "Regular"
            action = "N/A" if (i,j) == env.terminal_state else                     env.action_description[policy_standard[i, j]]

            summary_data.append({
                'State': f'({i},{j})',
                'Type': state_type,
                'Reward': env.reward[i, j],
                'V*(Standard)': round(V_standard[i, j], 3),
                'V*(In-Place)': round(V_inplace[i, j], 3),
                'Optimal Action': action
            })

    summary_df = pd.DataFrame(summary_data)
    display(summary_df)

    


## Talking Point:
I implemented and compared two variants of Value Iteration on a 5×5 gridworld with a goal state at (4,4) with +10 reward, three grey penalty states at positions (1,2), (3,0), and (0,4) with -5 rewards, and regular states with -1 rewards, using a discount factor γ = 0.9.

**Standard Value Iteration** converged in 9 iterations taking 0.003 seconds using synchronous updates with two value arrays.
**In-Place Value Iteration** also converged in 9 iterations but completed in 0.001 seconds—achieving a 3x speedup—by using a single array with asynchronous updates where values are immediately propagated.

Both algorithms produced identical optimal solutions with the same value function and policy. The optimal policy shows the agent navigating efficiently toward the goal, with most states choosing "right" (→) or "down" (↓) actions to create diagonal paths. Grey penalty states show significantly lower values (-7.44 to -8.10) compared to their neighbors, demonstrating the agent's learned avoidance behavior. For example, state (1,2) with the grey penalty has V* = -8.10, while adjacent regular states range from -4.69 to -3.44.

The key advantage of in-place VI is 50% memory reduction (requiring only one array versus two), which becomes critical for large state spaces. The 3x speedup comes from eliminating array copying overhead and achieving better cache locality. Both methods converge in the same number of iterations here, but in-place VI's immediate value propagation can sometimes accelerate convergence depending on state update ordering.

